# **SOLID**



## **S (SRP): Principio de Responsabilidad Única**

Cada clase debe tener una única responsabilidad o razón para cambiar.

**Problema**: Esta clase realiza dos tareas: generar un informe y guardarlo en un archivo. Esto viola el principio de responsabilidad única.

In [1]:
class ReporteManager:
    def __init__(self, data):
        self.data = data

    def generar(self):
        return f"Reporte Data: {self.data}"

    def guardar_archivo(self, filename):
        with open(filename, 'w') as file:
            file.write(self.generar())

**Mejora**: Ahora, las responsabilidades están divididas entre Report (generar informes) y FileManager (gestionar archivos).

In [2]:
class Reporte:
    def __init__(self, data):
        self.data = data

    def generar(self):
        return f"Reporte Data: {self.data}"

class ArchivoManager:
    def guardar_archivo(self, contenido, archivo):
        with open(archivo, 'w') as file:
            file.write(contenido)

# Uso:
reporte = Reporte("Estados Bancarios")
archivo = ArchivoManager()

archivo.guardar_archivo(reporte.generar(), "reporte.txt")

## **O (OCP): Principio Abierto/Cerrado**

**Problema**: Si necesitas agregar un nuevo tipo de descuento, tendrías que modificar el método `aplicar_decuento`, violando el principio abierto/cerrado.

In [3]:
class Descuento:
    def aplicar(self, precio, tipo_descuento):
        if tipo_descuento == "ninguno":
            return precio
        elif tipo_descuento == "porcentaje":
            return precio * 0.9
        elif tipo_descuento == "fijado":
            return precio - 10

**Mejora**: El sistema es extensible añadiendo nuevas clases de descuento sin modificar el código existente.

In [4]:
from abc import ABC, abstractmethod

class Descuento(ABC):
    @abstractmethod
    def aplicar(self, precio):
        pass


class NoDescuento(Descuento):
    def aplicar(self, precio):
        return precio


class PorcentajeDescuento(Descuento):
    def aplicar(self, precio):
        return precio * 0.9


class FijadoDiscount(Descuento):
    def aplicar(self, precio):
        return precio - 10


# Uso:
def calcular_precio(precio: int, descuento: Descuento):
    return descuento.aplicar(precio)

precio = 100
descuento = PorcentajeDescuento()

print(calcular_precio(precio, descuento))

90.0


## **L (LSP): Principio de Sustitución de Liskov**

**Problema**: La subclase `Avestruz` no puede comportarse como la clase base `Pajaro`, violando LSP.

In [5]:
class Pajaro:
    def volar(self):
        return "Volando"


class Avestruz(Pajaro):
    def volar(self):
        raise Exception("Las avestruces no pueden volar.")


# pajaro = Pajaro()
# pajaro.volar()

**Mejora**: Ahora, `PajaroVolador` y `PajaroNoVolador` representan comportamientos separados, evitando inconsistencias.

In [6]:
class Pajaro:
    pass


class PajaroVolador(Pajaro):
    def volar(self):
        return "Volando"


class PajaroNoVolador(Pajaro):
    def caminar(self):
        return "Caminando"


class Paloma(PajaroVolador):
    pass


class Avestruz(PajaroNoVolador):
    pass


# Uso
paloma = Paloma()
print(paloma.volar())

avestruz = Avestruz()
print(avestruz.caminar())
# print(avestruz.volar()) # AttributeError

Volando
Caminando


## **I (ISP): Principio de Segregación de Interfaces**

**Problema**: `ImpresoraBasica` está obligada a implementar métodos que no utiliza.

In [7]:
class Maquina:
    def imprimir_documento(self):
        pass

    def escanear_documento(self):
        pass

    def enviar_fax(self):
        pass


class ImpresoraBasica(Maquina):
    def imprimir_documento(self):
        return "Imprimiendo"

    def escanear_documento(self):
        raise NotImplementedError("No puede escanear")

    def enviar_fax(self):
        raise NotImplementedError("No puede enviar fax")

**Mejora**: Se han separado las interfaces para que las clases implementen únicamente lo que necesitan.

In [8]:
from abc import ABC, abstractmethod


class Impresora(ABC):
    @abstractmethod
    def imprimir_documento(self):
        pass


class Escaner(ABC):
    @abstractmethod
    def escanear_documento(self):
        pass


class ImpresoraBasica(Impresora):
    def imprimir_documento(self):
        return "Imprimiendo"


class ImpresoraTodoEnUno(Impresora, Escaner):
    def imprimir_documento(self):
        return "Imprimiendo"

    def escanear_documento(self):
        return "Escaneando"


# Uso
impresora_basica = ImpresoraBasica()
print(impresora_basica.imprimir_documento())
print()

impresora_todo = ImpresoraTodoEnUno()
print(impresora_todo.imprimir_documento())
print(impresora_todo.escanear_documento())

Imprimiendo

Imprimiendo
Escaneando


## **D (DIP): Principio de Inversión de Dependencias**

**Problema**: `ServicioNotificacion` depende directamente de `NotificacionEmail`. No es flexible ni reutilizable.

In [9]:
class NotificacionEmail:
    def enviar_email(self, mensaje):
        return f"Contenido: {mensaje}"


class ServicioNotificacion:
    def __init__(self):
        self.notificador = NotificacionEmail()

    def notificar(self, mensaje):
        return self.notificador.enviar_email(mensaje)


notificacion = ServicioNotificacion()
notificacion.notificar("Hola mucho gusto")

'Contenido: Hola mucho gusto'

Mejora: Ahora `ServicioNotificacion` depende de una abstracción `Notificador`, no de una implementación concreta.

In [10]:
from abc import ABC, abstractmethod


class Notificador(ABC):
    @abstractmethod
    def enviar(self, mensaje):
        pass


class NotificacionEmail(Notificador):
    def enviar(self, mensaje):
        return f"Contenido: {mensaje}"


class NotificacionSMS(Notificador):
    def enviar(self, mensaje):
        return f"Contenido: {mensaje}"


class ServicioNotificacion:
    def __init__(self, notificador: Notificador):
        self.notificador = notificador

    def notificar(self, mensaje):
        return self.notificador.enviar(mensaje)


# Uso:
notificador_email = NotificacionEmail()
notificador_sms = NotificacionSMS()

servicio = ServicioNotificacion(notificador_email)
print(servicio.notificar("Hola por correo"))

servicio = ServicioNotificacion(notificador_sms)
print(servicio.notificar("Hola por SMS"))


Contenido: Hola por correo
Contenido: Hola por SMS
